# ATML report
## Music genre recognition
Authors:
- Dorian Guyot
- Jonathan Péclat
- Thomas Schaller

Goal : ...

## Approach description


## Folder structure
```
ATML19
│   README.md : Simple readme for github
│   report.ipynb : Project report (this file)
│   report.html : Same as report.ipynb but in HTML format
│   pres.pdf : The pdf of the presentation
│
└─── notebooks : Notebook's files where tests were made
│   │   create_data_folder.ipynb : Create data folder when spectrogram are created
│   │   model_dorian.ipynb : Testing differents models on data
│   │   model_thomas.ipynb : Testing differents models on data
│   │   spectrogram.ipynb : Creating spectrograms from wav files
│   │   generate_experiments.ipynb : Generate a barplot from genre classification on external wav's musics.
│   │   src_python : Contains scripts use in generate_experiments (same as test_src)
│   
└─── test_src : Small app to test the final model
│   │   user_app.py : Main file of the app
│   └─── generate_data : Folder containing the file to create the spectogram of the music
│   │    │ spectrogram.py : Class to create the spectrogram of the music
│   │
│   └─── models : Folder containing files for the model
│   │    │ best_model_resnet : State dict of the best model created with resnet
│   │    │ model.py : Model of the project. Allow to predict genre of music
│   
└─── train_src : Small app to test the final model
│   │   main.py : Main file of the app to train the model
│   └─── generate_data : Folder containing files to process the data and create the data directory.
│   │    │ data_folder.py : Create the data folder to be able to use ImageFolder from pytorch then.
│   │    │ spectrogram.py : Create the spectrograms of all musics contain in a folder.
│   │
│   └─── model : Folder containing files for training the model
│   │    │ dataloader.py : Create the dataloader with the data of the data directory
│   │    │ model.py : train the model with the dataloader and save the best one
```

# Results
table results

## Example of use

Before using our work, you need to download some data. You can either download the wav and do the processing by ourself with our app, or directly download the data which were already processed. The data folder contains spectrograms of the wav files. Here is the link to download the two folders: https://www.dropbox.com/sh/dg1crj9yimefgpb/AADcOLk9fkLxFbaO7dn-rACDa?dl=0

## Generating experiments
For information, the code below can also be found in the notebook `generate_experiments`.

The folder containing the music to be processed below is located at the root of the project under the name `test_data`.

To begin with, we must make the necessary imports.

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
from test_src.generate_data.spectrogram import Spectrogram
from test_src.models.model import Model
%matplotlib inline

Then, we must list the music in the designated folder.

In [0]:
experiments_folder = 'test_data'
styles = ['blues','classical','country','disco','hiphop','jazz','metal','pop','reggae','rock']

In [0]:
test_data = []
for file in os.listdir(experiments_folder):
    if os.path.isdir(os.path.join(experiments_folder, file)):
        for file2 in os.listdir(os.path.join(experiments_folder, file)):
            if os.path.isfile(os.path.join(experiments_folder,file,file2)):
                test_data.append([file, file2])

Then we have to create the two classes that will be used to create the spectrograms as well as to predict the genre of the music. For the model, we will use the best model we have found, which is the one realized with transfer learning and resnet.

In [0]:
model_dict_path = "test_src/models/best_model_resnet"
model = Model()
model.load(model_dict_path)
spectrogram = Spectrogram()
length_data = len(test_data)

Finally, we will generate for each of the musics its corresponding spectrograms, and predict with the model what is the genre of the music. Then, to better visualize the result, a BarPlot was created with the percentage obtained for each of the genres.

In [0]:
i = 1
for data in test_data:
    print("Processing music ",i,"/",length_data)
    imgs = spectrogram.sample(os.path.join(experiments_folder,data[0],data[1]))
    results_sum = np.array([0.0] * len(styles))
    for img in imgs:
        results_sum += model.predict_image(img)
    results = results_sum / len(imgs) * 100.0
    y_pos = np.arange(len(styles))
    plt.bar(y_pos, results, align='center', alpha=0.5)
    plt.xticks(y_pos, styles, rotation=45)
    plt.ylim([0,100])
    plt.ylabel('Percent')
    plt.title('Title: '+data[1]+', True genre: '+data[0])
    plt.show()
    i += 1